In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Завантаження даних у DataFrame
data = pd.read_csv('hometask_5.csv')
print(data.head())

df = pd.DataFrame(data)

# Підготовка даних
# Перетворення 'Дата народження' у вік
df['Дата народження'] = pd.to_datetime(df['Дата народження'])
current_year = datetime.now().year
df['Вік'] = current_year - df['Дата народження'].dt.year

# Дискретизація 'Дохід на місяць'
bins = [0, 10000, 20000, 30000, 40000, 50000, np.inf]
labels = ['<10k', '10k-20k', '20k-30k', '30k-40k', '40k-50k', '>50k']
df['Дохід на місяць_кат'] = pd.cut(df['Доход на місяць'], bins=bins, labels=labels)

# Функція для обчислення IV
def calculate_iv(df, feature, target):
    # Створення крос-таблиці
    cross_tab = pd.crosstab(df[feature], df[target], margins=True)
    # Обчислення відсотків з обробкою ділення на нуль
    cross_tab['%_1'] = cross_tab[1] / (cross_tab[1]['All'] + 1e-10)
    cross_tab['%_0'] = cross_tab[0] / (cross_tab[0]['All'] + 1e-10)

    # Обчислення WOE та IV з обробкою логарифму
    cross_tab['WOE'] = np.log((cross_tab['%_1'] + 1e-10) / (cross_tab['%_0'] + 1e-10))
    cross_tab['IV'] = (cross_tab['%_1'] - cross_tab['%_0']) * cross_tab['WOE']

    # Видалення рядка 'All'
    cross_tab = cross_tab.drop('All', axis=0)

    # Сума IV
    iv = cross_tab['IV'].sum()
    return iv if not np.isnan(iv) else 0

# Список характеристик для аналізу
features = ['Вік', 'Дохід на місяць_кат', 'Тип зайнотості', 'Сімейний стан', 'Тип зайнятості', 'Місце проживання']

# Унікальні сегменти
segments = df['Сегмент'].unique()

# Обчислення IV для кожного сегменту
for segment in segments:
    print(f"\nСегмент: {segment} vs не {segment}")
    # Створення бінарної мітки для сегменту
    df['target'] = (df['Сегмент'] == segment).astype(int)

    # Обчислення IV для кожної характеристики
    for feature in features:
        if feature == 'Вік':
            # Дискретизація віку
            df['Вік_кат'] = pd.cut(df['Вік'], bins=[0, 20, 30, 40, 50, 60, 100], labels=['<20', '20-30', '30-40', '40-50', '50-60', '>60'])
            iv = calculate_iv(df, 'Вік_кат', 'target')
            print(f"  Вік (категорії): IV = {iv:.4f}")
        else:
            iv = calculate_iv(df, feature, 'target')
            print(f"  {feature}: IV = {iv:.4f}")


    CORDER  PERSONA              DATEVID  СУМА КРЕДИТУ  Оплата тіла кредиту  \
0  3035754  8551635  2021-04-13 09:51:57       10000.0              10000.0   
1  2949577   141820  2021-04-01 15:32:23       10000.0              10000.0   
2  3054276   400885  2021-04-15 14:16:27       10000.0              10000.0   
3  2953910  2050210  2021-04-02 08:38:04        8000.0               2004.0   
4  2979852  6778675  2021-04-05 14:04:19        6000.0               6000.0   

   Оплата відсотків  ФІНАНСОВИЙ РЕЗУЛТАТ      Дата народження  \
0           32409.0              32409.0  1982-12-09 00:00:00   
1           28736.0              28736.0  1990-08-13 00:00:00   
2           27050.0              27050.0  1997-09-15 00:00:00   
3           28516.0              22520.0  1979-03-12 00:00:00   
4           22036.2              22036.2  1981-05-01 00:00:00   

   Доход на місяць Тип зайнотості        Сімейний стан       Тип зайнятості  \
0          25000.0         Працюю  Не був (ла) у шлюбі 